In [18]:
from aocd.models import Puzzle
from typing import Tuple, Dict
from collections import defaultdict
from itertools import combinations, product, count
from functools import lru_cache
import re

puzzle = Puzzle(year=2020, day=12)
print(puzzle.input_data)

W2
F23
S1
W3
L180
W3
R90
N4
F17
S4
W4
R90
W4
E1
N4
F5
N2
R90
F43
N5
L90
F12
S4
W1
S3
W2
N4
F76
S1
W4
W2
F20
N4
F81
W4
N3
R180
W2
N4
W3
F16
N4
L180
F1
W3
F34
W3
S3
F92
L90
S2
L90
E5
N2
F85
W3
R90
W2
F88
N2
L90
W1
N4
L90
E5
N3
L90
F8
E1
L90
N3
F3
F61
S5
R90
W2
F84
W1
L90
E1
S3
W5
F89
E3
F67
E2
E5
F29
N5
W4
F53
N2
E5
F73
W4
L90
S2
R180
N3
R90
F27
N2
F41
L270
W5
F3
N5
F81
R90
N2
W5
N2
R90
S1
R180
S3
L90
E2
F38
S1
E3
S5
F44
N1
F26
E1
S2
F25
E1
S2
F33
S4
R90
N2
W2
F9
R90
F64
W1
S3
E5
R180
N2
L90
S4
E4
L90
S2
F50
S3
R90
F8
E2
N1
R90
S5
S1
F100
N3
F97
R180
S3
L180
F45
W5
S1
E5
E3
F26
N4
R90
N4
F50
W5
R90
F58
S4
W3
E1
N3
R90
S4
E2
F26
N4
L90
F60
W4
N1
F10
E5
L180
N4
E1
F15
E3
L270
F23
R90
F61
R90
F25
L90
W2
S3
R180
F44
W5
E1
R90
S2
R270
W2
S1
F13
E5
N2
R270
F68
F99
W1
F31
N5
F89
E5
N4
W5
N2
F59
E3
L180
E3
L90
L180
S5
F27
E1
S3
R180
N5
E4
L180
N2
E3
W1
L180
F23
N4
E1
F87
N1
E3
F45
W5
F17
L90
N1
L90
W3
S2
F62
R180
F8
R90
F19
W1
S2
S5
W4
F40
F52
S3
F6
R90
S5
W3
S5
E3
W3
S1
F11
S2
E4
F3
R90
L270
E1

In [19]:
moves = puzzle.input_data.splitlines()

deg2dir = {0:'N', 90:'E', 180:'S', 270:'W'}
deg2dir.update({y:x for x,y in deg2dir.items()})

def manhattan(pos: Tuple[int, int, int]) -> int:
    return abs(pos[0]) + abs(pos[1])

def move_NS(curr_pos:Tuple[int, int, int], amount:int) -> Tuple[int,int,int]:
    new = curr_pos[1]+amount
    return (curr_pos[0], new, curr_pos[2])

def move_EW(curr_pos:Tuple[int, int, int], amount:int) -> Tuple[int,int,int]:
    new = curr_pos[0]+amount
    return (new, curr_pos[1], curr_pos[2])

def vec_add(pos: Tuple[int, int], vec:Tuple[int,int]) -> Tuple[int,int]:
    return (pos[0]+vec[0], pos[1]+vec[1])

def move(instruction: str, curr_pos: Tuple[int, int, int]) -> Tuple[int, int, int]:
    com = instruction[0]
    amount = int(instruction[1:])
    if com == 'L':
        new_deg = (curr_pos[2]-amount)%360
        return (curr_pos[0], curr_pos[1], new_deg)
    if com == 'R':
        new_deg = (curr_pos[2]+amount)%360
        return (curr_pos[0], curr_pos[1], new_deg)
    if com == 'F':
        if curr_pos[2] == 0:
            return move_NS(curr_pos, amount)
        if curr_pos[2] == 180:
            return move_NS(curr_pos, -amount)
        if curr_pos[2] == 90:
            return move_EW(curr_pos, amount)
        if curr_pos[2] == 270:
            return move_EW(curr_pos, -amount)
    if com == 'E':
        return move_EW(curr_pos, amount)
    if com == 'W':
        return move_EW(curr_pos, -amount)
    if com == 'N':
        return move_NS(curr_pos, amount)
    if com == 'S':
        return move_NS(curr_pos, -amount)

def rotate_CCW(curr_pos: Tuple[int, int], amount:int) -> Tuple[int,int]:
    num_rots = amount//90
    for i in range(num_rots):
        curr_pos = (-curr_pos[1], curr_pos[0])
    return curr_pos

def rotate_CW(curr_pos: Tuple[int, int], amount:int) -> Tuple[int,int]:
    num_rots = amount//90
    for i in range(num_rots):
        curr_pos = (curr_pos[1], -curr_pos[0])
    return curr_pos

def solve_a(moves: list) -> int:
    curr_pos = (0, 0, 90)
    for i,instruction in enumerate(moves):
        curr_pos = move(instruction, curr_pos)
    return manhattan(curr_pos)

def solve_b(moves: list) -> int:
    curr_pos = (0, 0)
    waypoint = (10, 1)
    for move in moves:
        com = move[0]
        amount = int(move[1:])
        if com == 'N':
            waypoint = vec_add(waypoint, (0, amount))
        if com == 'S':
            waypoint = vec_add(waypoint, (0, -amount))
        if com == 'E':
            waypoint = vec_add(waypoint, (amount, 0))
        if com == 'W':
            waypoint = vec_add(waypoint, (-amount, 0))
        if com == 'L':
            waypoint = rotate_CCW(waypoint, amount)
        if com == 'R':
            waypoint = rotate_CW(waypoint, amount)
        if com == 'F':
            for i in range(amount):
                curr_pos = vec_add(curr_pos, waypoint)
    return sum(map(abs, curr_pos))

answer_a = solve_a(moves)
answer_b = solve_b(moves)

In [20]:
puzzle.answer_a = answer_a
puzzle.answer_b = answer_b

That's the right answer!  You are one gold star closer to saving your vacation.You have completed Day 12! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
